# Data Dictionary

|Attribute|Description
----------|-----------
InvoiceNo| Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
StockCode| Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
Description| Product (item) name. Nominal.
Quantity| The quantities of each product (item) per transaction. Numeric.
InvoiceDate| Invoice Date and time. Numeric, the day and time when each transaction was generated.
UnitPrice| Unit price. Numeric, Product price per unit in sterling.
CustomerID| Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
Country| Country name. Nominal, the name of the country where each customer resides.

# 0.0 Imports

In [78]:
import pandas as pd
import inflection
from src import GeneralUtils  as gu
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.cm as cm
import numpy as np
import scipy.stats as st
from scipy.stats import kstest

import umap.umap_ as um
from sklearn.decomposition import PCA
import sklearn.metrics as mt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import sklearn.preprocessing as pp
import sklearn.manifold as man

from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
import sklearn.mixture as mix
from scipy.cluster.hierarchy import dendrogram, ward, linkage, fcluster

import dotenv
import datetime as dt
import sqlite3
from sqlalchemy import create_engine

# 0.1 Load Data

In [79]:
secret = dotenv.dotenv_values('../vars/.env')['SECRET_KEY']
key = dotenv.dotenv_values('../vars/.env')['ACCESS_KEY']

path_s3 = 's3://insider-data-bucket/'
df_raw = pd.read_csv(path_s3 +  'Ecommerce.csv', 
                     encoding='ISO-8859-1',
                     storage_options={'key': key,
                                      'secret': secret})
#df_raw = pd.read_csv('../data/raw/Ecommerce.csv', encoding='ISO-8859-1')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 1.0. Data Description

In [80]:
df_description = df_raw.drop('Unnamed: 8', axis=1)

## 1.1. Renaming Columns

In [81]:
old_cols = ['InvoiceNo', 'StockCode', 'Description', 'Quantity','InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore(x)
new_cols = list(map(snakecase, old_cols))

df_description.columns = new_cols

In [82]:
df_description['description'] = df_description['description'].str.lower()

## 1.4.  Handling Missing Values

**I will add a custom customer ID for each unique inovice number with a missing customer ID**

Issue: Doing this, these new customers' IDs will only have one transaction, but this might be better than removing all this data.

In [83]:
df_missing = df_description.query('customer_id.isnull()')
missing_invoice_no = df_missing[['invoice_no']].drop_duplicates()
missing_invoice_no['customer_id'] = np.arange(19000, 19000+len(missing_invoice_no))
df_description = df_description.merge(missing_invoice_no, on='invoice_no', how='left')
df_description['customer_id'] = df_description['customer_id_x'].combine_first(df_description['customer_id_y'])
df_clean = df_description.drop(columns=['customer_id_x', 'customer_id_y'])

## 1.5. Fixing Data Types

In [84]:
df_clean.invoice_date = pd.to_datetime(df_clean.invoice_date)
df_clean.customer_id = df_clean.customer_id.astype(int)

## 1.6. Descriptive Statistics

### 1.6.1 Numerical Attributes

In [85]:
num_att = df_clean.select_dtypes(include=['float64', 'int64'])
cat_att = df_clean.select_dtypes(exclude=['float64', 'int64', 'datetime64[ns]'])

### 1.6.2 Categorical Attributes

**Invoice Number with at least one string**

In [86]:
# df_description.inoice_no.astype(int)
letter_invoices = df_clean.loc[df_description.invoice_no.str.contains('C')]

print('Total Number of Cancellation invoices:', len(letter_invoices))
print('Total Number of Negative quantites:', len(letter_invoices.query('quantity < 0')))

Total Number of Cancellation invoices: 9288
Total Number of Negative quantites: 9288


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 2.0. Variables Filtering

In [87]:
df_filter = df_clean.copy()

## 2.1. Numerical

By the description of the items with a negative quantity and are not cancellations, it seems like they are some mistakes or an issue with the product. Since these instances don't represent actual sales, we will remove them from the dataset.

In [88]:
df_filter = df_filter.query("~(quantity <= 0 & invoice_no.str.isdigit())")

**Unit prices very close to zero**

Half of the instances don't have a description, which makes it harder to define what these instances mean. Therefore, they will be removed (unit price < 0.004).

In [89]:
df_filter = df_filter.query('unit_price >= 0.004')

## 2.2. Categorical

In [90]:
# removing stock_codes that are only letters
df_filter = df_filter.query("~stock_code.isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])")

# removing European Community and Unspecified
df_filter = df_filter.query("~country.isin(['European Community', 'Unspecified'])")

country_dict = {'EIRE': 'Ireland', 'RSA': 'South Africa', 'USA': 'United States', 'Channel Islands': 'United Kingdom'}

countries_to_modify = ['USA', 'Channel Islands', 'RSA', 'EIRE']

df_filter.country = df_filter.country.apply(lambda x: country_dict[x] if x in countries_to_modify else x)

In [91]:
# description
print(f'Removed data: {1 - (df_filter.shape[0]/df_clean.shape[0]):.3%}')
print(f'Number of Elements Removed: {df_clean.shape[0] - df_filter.shape[0]}')

Removed data: 1.064%
Number of Elements Removed: 5765


In [92]:
print('Number of rows Cancelation:', df_filter.query("quantity < 0 or invoice_no.str.contains('C')").shape[0])
print('Number of rows Transaction:', df_filter.query('quantity > 0').shape[0])

Number of rows Cancelation: 8730
Number of rows Transaction: 527414


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 3.0. Feature Engineering

In [93]:
df_feature = df_filter.copy()
df_feature_cancelation = df_filter.query("quantity < 0 or invoice_no.str.contains('C')").copy()
df_feature_transaction = df_filter.query('quantity > 0').copy()

In [94]:
df_ref = df_feature.drop_duplicates('customer_id')

**Latitude and Longitude of the countries**

In [95]:
countries_lat_long = pd.read_csv('../data/external/countries.csv')
countries_lat_long.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [96]:
countries_lat_long.rename(columns={'name': 'country', 'country': 'code'}, inplace=True)

**Recency, Frequency and Monetary**

In [97]:
df_feature['gross_revenue'] = df_feature.quantity * df_feature.unit_price

# Recency
diff = df_filter.invoice_date.max() - df_feature_transaction.groupby('customer_id').max()['invoice_date']
recency = pd.DataFrame(diff).reset_index().rename(columns={'invoice_date': 'recency'})

recency['recency'] = recency['recency'].dt.days
df_ref = df_ref.merge((recency), how='left', on='customer_id')


In [98]:
# Frequency
min_date = df_feature.invoice_date.min()
max_date = df_feature.invoice_date.max()
aux = df_feature_transaction.groupby('customer_id').agg( max_ = ('invoice_date', 'max'), 
                                       min_ = ('invoice_date', 'min'),
                                       #days_ = ('invoice_date', lambda x: (max_date - min_date).days + 1),
                                       days_ = ('invoice_date', lambda x: (max_date - x.min()).days + 1),
                                       buy_ = ('invoice_no', 'nunique'),
                                      ).reset_index()

aux['frequency'] = aux.buy_/aux.days_
aux['frequency'] = aux.buy_/aux.days_

df_ref = df_ref.merge(aux[['customer_id', 'frequency']], on='customer_id', how='left')

In [99]:
# Total Spent
monetary = df_feature.groupby('customer_id').sum(numeric_only=True).gross_revenue.reset_index().rename(columns={'gross_revenue': 'total_spent'})
df_ref = df_ref.merge((monetary), how='left', on='customer_id')

**Number of transactions, Variety, Average Ticket and Average Basket Size**

In [100]:
# Transactions
transactions = df_feature.groupby('customer_id').nunique()[['invoice_no']].reset_index().rename(columns={'invoice_no': 'transactions'})
df_ref = df_ref.merge((transactions), how='left', on='customer_id')

In [101]:
# Average Ticket
avg_ticket = df_feature.groupby(['customer_id', 'invoice_no']).sum(numeric_only=True).gross_revenue.reset_index().rename(columns={'gross_revenue': 'avg_ticket'}).groupby('customer_id').mean(numeric_only=True).reset_index()
df_ref = df_ref.merge((avg_ticket), how='left', on='customer_id')

In [102]:
# Product Variety
product_variety = df_feature_transaction.groupby('customer_id').nunique()[['description']].reset_index().rename(columns={'description': 'product_variety'})
df_ref = df_ref.merge((product_variety), how='left', on='customer_id')

In [103]:
# Average Basket Size 
avg_basket = df_feature[['customer_id', 'invoice_no', 'quantity']].groupby(['customer_id', 'invoice_no']).sum()\
                                                                                                                .reset_index().groupby('customer_id').mean(numeric_only=True)\
                                                                                                                .reset_index().rename(columns={'quantity': 'avg_basket'})
df_ref = df_ref.merge((avg_basket), how='left', on='customer_id')

In [104]:
# Average Basket Variety Size
avg_basket_var = df_feature_transaction[['customer_id', 'invoice_no', 'stock_code']].groupby(['customer_id', 'invoice_no']).nunique()\
                                                                                                                .reset_index().groupby('customer_id').mean(numeric_only=True)\
                                                                                                                .reset_index().rename(columns={'stock_code': 'avg_basket_variety'})
df_ref = df_ref.merge(avg_basket_var, how='left', on='customer_id')

**Average Recency, Total Cancelltions, Quantity** - <font color='red'>Average Recency Removed</font>

In [105]:
cancelations = df_feature_cancelation[['customer_id', 'quantity']].groupby('customer_id').apply(lambda c: c.abs().sum())[['quantity']].reset_index().rename(columns={'quantity': 'number_of_cancellations'})
df_ref = df_ref.merge(cancelations, how='left', on='customer_id')

In [106]:
# Total Quantity
quantity = df_feature.groupby('customer_id').sum(numeric_only=True).quantity.reset_index().rename(columns={'quantity': 'total_quantity'})
df_ref = df_ref.merge(quantity, how='left', on='customer_id')

Some customers only have cancellations, they need to be removed.

In [108]:
columns = ['customer_id', 'country', 'recency', 'frequency', 'total_spent', 'transactions', 'total_quantity', 'avg_ticket', 'product_variety', 'avg_basket', 'avg_basket_variety', 'number_of_cancellations']
df_customers = df_ref.drop_duplicates('customer_id').copy()[columns]
df_customers.head()

,customer_id,country,recency,frequency,total_spent,transactions,total_quantity,avg_ticket,product_variety,avg_basket,avg_basket_variety,number_of_cancellations
0,17850,United Kingdom,372.0,0.090909,5288.63,35,1693,151.103714,21.0,48.371429,8.735294,40.0
1,13047,United Kingdom,56.0,0.024064,3089.10,16,1355,193.068750,105.0,84.687500,19.000000,35.0
2,12583,France,2.0,0.040107,6629.34,17,4978,389.961176,116.0,292.823529,15.466667,50.0
3,13748,United Kingdom,95.0,0.013369,948.25,5,439,189.650000,24.0,87.800000,5.600000,NaN
4,15100,United Kingdom,333.0,0.008021,635.10,6,58,105.850000,1.0,9.666667,1.000000,22.0


The number of cancelations with null values are customers with no cancellations.

I want to focus on customers who have made purchases. Therefore I will remove those with a negative monetary values, which may mean they have more canceled orders than purchases.

In [110]:
# fill null values for number of cancellations
df_customers['number_of_cancellations'] = df_customers['number_of_cancellations'].fillna(0)
df_customers_clean = df_customers.query('(total_spent > 0)')

print('Items Removed:', df_customers.shape[0] - df_customers_clean.shape[0])

Items Removed: 101


After filtering the dataset, I will add the longitude and latitude for the countries.

In [112]:
df_customers_clean = df_customers_clean.merge(countries_lat_long, on='country', how='left')

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 4.0. Exploratory Data Analysis

In [113]:
df_eda = df_customers_clean.copy().drop(columns='code', axis=0)

**Removing bad users**

In [114]:
df_eda = df_eda.query('~(customer_id == 16446)').copy()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 5.0. Data Preparation

In [115]:
df_prep = df_eda.copy()

In [116]:
std = Pipeline([('standard', pp.StandardScaler())])
minmax = Pipeline([('minmax', pp.MinMaxScaler())])
robust = Pipeline([('robust', pp.RobustScaler())])
cols = ['recency', 'frequency', 'total_spent', 'total_quantity', 'transactions', 'avg_ticket', 'product_variety', 'avg_basket', 'number_of_cancellations', 'latitude', 'longitude']
pass_cols = ['customer_id'] 

preprocess = ColumnTransformer([('pass', 'passthrough', pass_cols), ('minmax', minmax, cols)])
#std_preprocess = ColumnTransformer([('pass', 'passthrough', pass_cols), ('standard', std, cols)])

**Using Min Max Scaler for all features.**

In [117]:
#df_prep_scaled
df_prep_scaled = pd.DataFrame(preprocess.fit_transform(df_prep))
cols = np.concatenate((pass_cols, cols), axis=0)
df_prep_scaled.columns = cols

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 6.0. Feature Selection

In [118]:
# remove avg_basket
# remove avg_recency
# remove items
# remove Transaction

columns_selected = ['customer_id', 'recency', 'frequency', 'total_spent', 'total_quantity', 'number_of_cancellations']

In [119]:
df_fselect = df_prep.copy()[columns_selected]
df_fselect_sc = df_prep_scaled.copy()[columns_selected]

# 7.0. Embedding Space

In [120]:
df_embed = df_fselect.drop(columns=['customer_id']).copy()

df_embed_sc = df_fselect_sc.drop(columns=['customer_id']).copy()

## 7.4. Tree-Based Embedding

Variable that organize the customers in a way that I can group the most valuable clients.

In [121]:
X = df_embed.drop(columns='total_spent')
y = df_embed['total_spent']

# model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
# training
rf.fit(X, y)

# Aplying the tree structure in the training set (Leafs)
df_leaf = pd.DataFrame(rf.apply(X)) 


In [122]:
umap = um.UMAP(n_neighbors=50, random_state=42, n_jobs=-1, n_epochs=500, n_components=4)

df_umap_rf = umap.fit_transform(df_leaf)
df_embed_tree = pd.DataFrame(df_umap_rf)
#plt.figure(figsize=(12, 6))

#sns.scatterplot(x=df_umap_rf[:, 0], y=df_umap_rf[:, 1])


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 8.0. Hyperparameter Fine-Tuning

In [123]:
#X = df_fselect_sc.copy().reset_index(drop=True).drop(columns=['customer_id'])
# Using the data of the embedding space build with umap and a tree model

X = df_embed_sc.copy()

X_tree = df_embed_tree.copy()

#X_umap = df_umap_sc.copy()
#
#X_tsne = df_tsne_sc.copy()

# Results

In [124]:
#clusters = np.arange(2, 30, 1)
#hierarch_perf = gu.hierarchical_performance(X=X_tree, clusters=clusters, plot=False, metric='euclidean')
#gmm_perf = gu.gmm_performance(X=X_tree, components=clusters, plot=False, metric='euclidean', covariance_type='full')
#kmeans_perf = gu.kmeans_performance(X=X_tree, clusters=clusters, plot=False, metric='euclidean')
#
#results = pd.concat([kmeans_perf, hierarch_perf, gmm_perf])
#results.columns = clusters
#
#results.style.highlight_max(color='lightgreen', axis=1)

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
KMeans,0.291570,0.382762,0.432783,0.501478,0.546204,0.613159,0.667090,0.680232,0.711895,0.721101,0.699584,0.693100,0.672692,0.709694,0.672105,0.683605,0.665786,0.652746,0.649541,0.652423,0.656381,0.659273,0.646977,0.658203,0.657271,0.666722,0.640775,0.651493
H_cluster,0.281756,0.360921,0.436901,0.493848,0.552678,0.613159,0.663458,0.695218,0.705061,0.721738,0.730329,0.717533,0.720402,0.718992,0.704435,0.699930,0.691559,0.671461,0.661395,0.673444,0.661212,0.668865,0.669700,0.678976,0.677377,0.682321,0.678859,0.672535
Gaussian Mixture,0.202114,0.326373,0.377151,0.446171,0.512849,0.602088,0.600210,0.630878,0.705061,0.699691,0.681721,0.701971,0.668629,0.680112,0.670136,0.649052,0.664332,0.645093,0.653155,0.666121,0.656413,0.650888,0.664450,0.664628,0.671636,0.662933,0.672381,0.663529


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 8.0. Model Training

## 8.1. Hierarchical Clustering

In [81]:
n_cluster = 11
agg_clu = AgglomerativeClustering(n_clusters=n_cluster)

labels = agg_clu.fit_predict(X_tree)

sil = mt.silhouette_score(X_tree, labels, metric='euclidean')

print(f'Silhouette Score: {sil}')

Silhouette Score: 0.7217379808425903


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 9.0. Cluster Analysis

## 9.1. Visualization

In [82]:
cluster_df = df_fselect.copy()
cluster_df_sc = df_fselect_sc.copy()
cluster_df['cluster'] = labels
cluster_df_sc['cluster'] = labels

## 9.2. Clusters Profile

In [86]:
prof = cluster_df.groupby('cluster').count()['customer_id'].reset_index().rename(columns={'customer_id': 'number_of_customers'})
prof['percentual'] = round((cluster_df.groupby('cluster').count().reset_index()['customer_id']/cluster_df.shape[0])*100, 2)

prof['avg_recency'] = cluster_df.groupby('cluster').mean().reset_index()['recency']

prof['avg_frequency'] = cluster_df.groupby('cluster').mean().reset_index()['frequency']

prof['avg_spent'] = cluster_df.groupby('cluster').mean().reset_index()['total_spent']

prof['number_of_cancellations'] = cluster_df.groupby('cluster').mean().reset_index()['number_of_cancellations']

prof['avg_quantity'] = cluster_df.groupby('cluster').mean().reset_index()['total_quantity']


prof

,cluster,number_of_customers,percentual,avg_recency,avg_frequency,avg_spent,number_of_cancellations,avg_quantity
0,0,544,9.57,204.505515,0.015439,66.714798,0.176471,17.108456
1,1,917,16.13,118.008724,0.019063,589.918364,8.735005,262.605234
2,2,627,11.03,119.998405,0.016655,1004.995566,3.945774,427.122807
3,3,695,12.23,144.263309,0.017046,340.175252,1.103597,144.633094
4,4,677,11.91,179.468242,0.016562,232.314402,0.660266,73.425406
5,5,487,8.57,30.172485,0.052795,10708.476448,162.745380,6309.527721
6,6,592,10.41,49.010135,0.034473,2682.552196,15.003378,1422.554054
7,7,655,11.52,77.419847,0.022749,1605.094122,13.833588,844.632061
8,8,113,1.99,210.504425,0.010335,5.755221,0.000000,1.000000
9,9,302,5.31,63.923841,0.024744,1144.909735,7.437086,559.993377


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


# 11.0. Deploy to Production

In [87]:
cluster_df['last_training_time'] = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# Banco NA AWS
variables_env = dotenv.dotenv_values('../vars/.env')

host = variables_env['HOST']
port = variables_env['PORT']
database =variables_env['DATABASE']
user=variables_env['USER']
pwd=variables_env['PASSWORD']

endpoint = f'postgresql+psycopg2://{user}:{pwd}@{host}:{port}/postgres'

engine = create_engine(endpoint)
con = engine.connect()

In [ ]:
query_create_table_insiders = """
    CREATE TABLE IF NOT EXISTS insiders(
        customer_id                 INTEGER,
        recency                     INTEGER,
        frequency                   REAL,
        total_spent                 REAL,
        total_quantity              INTEGER,
        number_of_cancellations     INTEGER,
        cluster                     INTEGER,
        last_training_time          TEXT
    )"""

# Create Table

con.execute(query_create_table_insiders)

# Inser Data

#cluster_df.to_sql('insiders', con=con, if_exists='append', index=False)


In [ ]:
test = pd.read_sql('SELECT * FROM insiders', con=con)

combined = pd.concat([test, cluster_df], axis=0)
cols = combined.drop(columns='last_training_time').columns.to_list()
combined = combined.drop_duplicates(subset=cols, keep='last')

combined.to_sql('insiders', con=con, if_exists='replace', index=False)

con.close()